In [97]:
import sys
sys.path.insert(1, '../Src')
import numpy as np
import pandas as pd
import os
from utils.feature_engineering import *
from utils.preprocessing import *
from utils.analysis import * 
PATH_DATASET = '../Dataset/'
#PATH_DATASET = '../Dataset/datah-m1-challange/'
PATH_RESULTS = '../Results/'

In [98]:
df_sales  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','sales.pkl'))
df_items  = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','item.pkl'))
df_stores = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','store.pkl'))

df_submission_sample = pd.read_pickle(os.path.join(PATH_RESULTS,'dataset','submission_sample.pkl'))

In [99]:
df_sales.head()

,DATE,S100,I100,C100,C101,QTT,item_id
0,2017-01-08,0,0,12,76,2,0_12_76
1,2017-01-08,0,0,12,149,3,0_12_149
2,2017-01-08,0,0,12,256,3,0_12_256
3,2017-01-08,0,0,12,354,3,0_12_354
4,2017-01-08,0,0,13,149,2,0_13_149


In [100]:
N_submission = df_submission_sample.shape[0]
N_sales      = df_sales.shape[0]

In [101]:
df_sales = fe_dates(df_sales)
df_submission_sample = fe_dates(df_submission_sample)

In [102]:
df_sales = df_sales.merge(df_items,on=['I100'])
df_sales = df_sales.merge(df_stores,on=['S100'])
    
df_submission_sample = df_submission_sample.merge(df_items,on=['I100'])
df_submission_sample = df_submission_sample.merge(df_stores,on=['S100'])



In [103]:
def string_to_categorical(df):
    for name in df.columns:
        aux = df[name].dtype
        if str(aux) in ['str','object']:
            df[name] = df[name].astype('category')
            print(name,aux,df[name].dtype)
    return df

In [104]:
df_sales.replace([np.inf, -np.inf, np.nan],0,inplace=True)
df_submission_sample.replace([np.inf, -np.inf, np.nan],0,inplace=True)
df_sales             = string_to_categorical(df_sales).copy()
df_submission_sample = string_to_categorical(df_submission_sample).copy()


item_id object category
ID object category
item_id object category


In [105]:
df_sales.head()

,DATE,S100,I100,C100,C101,QTT,item_id,year,month,day,...,is_year_start,is_quarter_start,is_month_start,is_month_end,I101,I102,I103,S101,S102,S103
0,2017-01-08,0,0,12,76,2,0_12_76,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
1,2017-01-08,0,0,12,149,3,0_12_149,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
2,2017-01-08,0,0,12,256,3,0_12_256,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
3,2017-01-08,0,0,12,354,3,0_12_354,2017,1,8,...,False,False,False,False,2,1,1,1,17,10
4,2017-01-08,0,0,13,149,2,0_13_149,2017,1,8,...,False,False,False,False,2,1,1,1,17,10


In [106]:
def categorical_to_numeric(df):
    cat_columns = df.select_dtypes(['category']).columns
    print(cat_columns)
    df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
    df[cat_columns] = df[cat_columns].astype(int)
    return df

In [107]:
print(df_sales.shape,df_submission_sample.shape)
df_auxiliar = pd.concat([df_sales,df_submission_sample])
df_auxiliar.replace([np.inf, -np.inf, np.nan],0,inplace=True)

df_auxiliar = string_to_categorical(df_auxiliar).copy()

dates = df_auxiliar['DATE'].unique()

date = df_auxiliar['DATE'].min()
maxi = df_auxiliar['DATE'].max()

dict_dates = {}
idx = 0
while(True):    
    if date>maxi:
        break
    dict_dates[date]=idx
    date = date+ pd.DateOffset(days=7)    
    idx = idx+1
    
df_auxiliar['date_block_num'] = df_auxiliar['DATE'].replace(dict_dates)

for value in ["S100","S101", "S102","S103","I100","I101","I102","I103","C100","C101"]:        
    df_auxiliar[value] = df_auxiliar[value].astype(str).astype(int)
    
df_auxiliar = categorical_to_numeric(df_auxiliar)

#df_auxiliar = columns_to_str(df_auxiliar,["S101", "S102","S103","I100","I101","I102","I103","C100","C101"])



(2749369, 22) (69121, 23)
item_id object category
Index(['item_id', 'ID'], dtype='object')


In [108]:
df_sales, df_submission_sample = df_auxiliar[:N_sales], df_auxiliar[N_sales:]
print(df_sales.shape,df_submission_sample.shape)

(2749369, 24) (69121, 24)


#'day_of_week', 'day', 'is_month_end', 'day_of_year',  'is_quarter_start', 'year', 'month', 'is_year_start', 'is_month_start'
for cat_col in ["S100", "item_id","S101", "S102","S103","I100","I101","I102","I103","C100","C101"]:
    df_sales[cat_col]             = (df_sales[cat_col].astype(str) +'_'+ cat_col).astype('category')
    df_submission_sample[cat_col] = (df_submission_sample[cat_col].astype(str) +'_'+ cat_col).astype('category')


In [109]:
df_sales.replace(['',np.inf, -np.inf, np.nan],0,inplace=True)
df_submission_sample.replace(['',np.inf, -np.inf, np.nan],0,inplace=True)


/opt/conda/lib/python3.8/site-packages/pandas/core/frame.py:5238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [110]:
date_block_num_min_sales = df_sales['date_block_num'].min()
date_block_num_max_sales = df_sales['date_block_num'].max()
print(date_block_num_min_sales,date_block_num_max_sales)

aux = df_sales['DATE'].min()
id_aux = 0
for i in range(date_block_num_min_sales,date_block_num_max_sales+1):
    ga = df_sales[df_sales['date_block_num']==i]
    if len(ga)>0:
        date = ga['DATE'].iloc[0]
        #print(date,i)
        if date<aux :
            print('discontinuidad')
            print('before :',aux,id_aux)
            print('actual :',date,i)
        elif date==aux and id_aux !=0:
            print('igualdad')
            print('before :',aux,id_aux)
            print('actual :',date,i)
        else:
            pass
        aux = date
        id_aux =i
    else:
        print('date before',id_aux,aux)
        print('date not exists',i)

0 246
date before 167 2020-03-22 00:00:00
date not exists 168
date before 167 2020-03-22 00:00:00
date not exists 169
date before 167 2020-03-22 00:00:00
date not exists 170
date before 167 2020-03-22 00:00:00
date not exists 171


In [111]:
date_block_num_min_sales = df_auxiliar['date_block_num'].min()
date_block_num_max_sales = df_auxiliar['date_block_num'].max()
print(date_block_num_min_sales,date_block_num_max_sales)

aux = df_auxiliar['DATE'].min()
id_aux = 0
for i in range(date_block_num_min_sales,date_block_num_max_sales+1):
    ga = df_auxiliar[df_auxiliar['date_block_num']==i]
    if len(ga)>0:
        date = ga['DATE'].iloc[0]
        #print(date,i)

0 254


In [112]:
features_names = set(df_sales.columns)-set(['QTT','DATE','ID'])
print(features_names)

{'I100', 'date_block_num', 'day_of_year', 'is_quarter_start', 'day', 'S103', 'S100', 'C101', 'is_month_end', 'I101', 'day_of_week', 'year', 'month', 'is_year_start', 'I102', 'is_month_start', 'S102', 'S101', 'I103', 'item_id', 'C100'}


In [113]:
df_submission_sample['DATE'].min(),df_submission_sample['DATE'].max()

(Timestamp('2021-10-03 00:00:00'), Timestamp('2021-11-21 00:00:00'))

In [114]:
#features_names = ['DATE','QTT','date_block_num',"S100", "item_id","S101", "S102","S103","I100","I101","I102","I103","C100","C101",'day_of_week', 'day', 'is_month_end', 'day_of_year',  'is_quarter_start', 'year', 'month', 'is_year_start', 'is_month_start']

In [115]:
df_sales['QTT'] = df_sales['QTT']*1.0

/tmp/ipykernel_4971/3293773272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sales['QTT'] = df_sales['QTT']*1.0


In [116]:
df_sales.head(2)

,DATE,S100,I100,C100,C101,QTT,item_id,year,month,day,...,is_month_start,is_month_end,I101,I102,I103,S101,S102,S103,ID,date_block_num
0,2017-01-08,0,0,12,76,2.0,4,2017,1,8,...,False,False,2,1,1,1,17,10,-1,0
1,2017-01-08,0,0,12,149,3.0,0,2017,1,8,...,False,False,2,1,1,1,17,10,-1,0


In [117]:
import numpy as np

In [118]:
import lightgbm as lgb

hyper_params ={      
  'boosting_type': 'gbdt',
   'objective': 'tweedie',
   'tweedie_variance_power': 1.1,
    'metric': 'rmse',
    'subsample': 0.6,
    'subsample_freq': 1,
    'learning_rate': 0.02,
    'num_leaves': 128,
    'min_data_in_leaf':256,
    'feature_fraction': 0.6,
    'max_bin': 100,
    #'n_estimators': …,   
    'boost_from_average': False,
    'verbose': -1,
    'num_threads': 12
   }    


In [142]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

model_version = 'model_v1'
unique_stores = sorted(df_submission_sample['S100'].unique())

predictions_store = {}

store = None
#for store in unique_stores:
print('*'*10,store,'*'*10)
print('Analyzing store :',store)

df_sales2             = df_sales #[df_sales['S100']==store]
df_submission_sample2 = df_submission_sample#[df_submission_sample['S100']==store]

X_train      = df_sales2[(df_sales2['DATE'] >= '2018-01-01') & (df_sales2['DATE'] < '2021-08-01')].copy()
X_validation = df_sales2[(df_sales2['DATE'] >= '2021-08-01') & (df_sales2['DATE'] < '2021-10-01')].copy()
X_test       = df_sales2[(df_sales2['DATE'] >= '2021-08-01') & (df_sales2['DATE'] < '2021-10-03')].copy() 

y_train = X_train['QTT']
X_train = X_train[features_names]

y_validation = X_validation['QTT']
X_validation = X_validation[features_names]

y_test = X_test['QTT']
X_test = X_test[features_names]

y_submission = df_submission_sample2['QTT']
X_submission = df_submission_sample2[features_names]


print('X_train      ',X_train.shape)
print('X_validation ',X_validation.shape)
print('X_test       ',X_test.shape)
print('X_submission ',X_submission.shape)

lgb_train = lgb.Dataset(X_train, y_train,categorical_feature =  ["S100","S101", "S102","S103","I100","I101","I102","I103","C100","C101"])
lgb_valid = lgb.Dataset(X_validation, y_validation,categorical_feature =  ["S100","S101", "S102","S103","I100","I101","I102","I103","C100","C101"])

gbm2 = lgb.train(hyper_params, lgb_train, 3000, valid_sets=[lgb_train, lgb_valid],early_stopping_rounds=50, verbose_eval=50)





print('make predictions...')
X_train_pred      = gbm2.predict(X_train)#, num_iteration=gbm.best_iteration_)
X_validation_pred = gbm2.predict(X_validation)#, num_iteration=gbm.best_iteration_)
X_test_pred       = gbm2.predict(X_test)#, num_iteration=gbm.best_iteration_)

X_submission_pred = gbm2.predict(X_submission)#, num_iteration=gbm.best_iteration_)


X_train_pred = np.maximum(X_train_pred, 0)
X_validation_pred = np.maximum(X_validation_pred, 0)
X_test_pred = np.maximum(X_test_pred, 0)
X_submission_pred = np.maximum(X_submission_pred, 0)

df_train_rmse = mean_squared_error(y_train,X_train_pred, squared=False)
df_val_rmse   = mean_squared_error(y_validation,X_validation_pred, squared=False)
df_test_rmse  = mean_squared_error(y_test,X_test_pred, squared=False)

print('Final score mean_squared_error')
print('Score train:',df_train_rmse)
print('Score val  :',df_val_rmse)
print('Score test :',df_test_rmse)


#model_name = os.path.join(PATH_RESULTS,'lightgbm',model_version,'model_'+str(store)+'.txt')
#print('saving model..',model_name)
#gbm2.save_model(model_name, num_iteration=gbm2.best_iteration)

predictions_store[store] = X_submission_pred



********** None **********
Analyzing store : None
X_train       (1980809, 21)
X_validation  (95515, 21)
X_test        (95515, 21)
X_submission  (69121, 21)
Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.62968	valid_1's rmse: 3.33872
[100]	training's rmse: 2.34619	valid_1's rmse: 3.11456
[150]	training's rmse: 2.23402	valid_1's rmse: 3.03131
[200]	training's rmse: 2.17758	valid_1's rmse: 2.99589
[250]	training's rmse: 2.14164	valid_1's rmse: 2.97686
[300]	training's rmse: 2.11516	valid_1's rmse: 2.96554
[350]	training's rmse: 2.09325	valid_1's rmse: 2.95895
[400]	training's rmse: 2.0748	valid_1's rmse: 2.95419
[450]	training's rmse: 2.05949	valid_1's rmse: 2.95051
[500]	training's rmse: 2.04632	valid_1's rmse: 2.9479
[550]	training's rmse: 2.03299	valid_1's rmse: 2.94533
[600]	training's rmse: 2.02155	valid_1's rmse: 2.94357
[650]	training's rmse: 2.01032	valid_1's rmse: 2.94222
[700]	training's rmse: 2.00219	valid_1's rmse: 2.941
[750]	training's r

In [143]:
import xgboost as xgb

model = xgb.XGBRegressor(learning_rate=0.01,
                       n_estimators=600,
                       max_depth=4,
                       min_child_weight=0,
                       gamma=0.6,
                       subsample=0.7,
                       colsample_bytree=0.7,
                       objective='reg:linear',
                       nthread=-1,
                       scale_pos_weight=1,
                       seed=27,
                       reg_alpha=0.00006,
                       random_state=42)

In [144]:
model.fit(X_train, y_train,
       eval_set = [(X_train, y_train), (X_validation, y_validation)],
       early_stopping_rounds = 10,
       eval_metric='rmse',
       verbose = True)

[01:38:48] WARNING: ../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:3.54506	validation_1-rmse:4.13556
[1]	validation_0-rmse:3.53179	validation_1-rmse:4.12321
[2]	validation_0-rmse:3.51993	validation_1-rmse:4.11257
[3]	validation_0-rmse:3.50861	validation_1-rmse:4.10180
[4]	validation_0-rmse:3.49708	validation_1-rmse:4.09124
[5]	validation_0-rmse:3.48508	validation_1-rmse:4.07946
[6]	validation_0-rmse:3.47173	validation_1-rmse:4.06811
[7]	validation_0-rmse:3.45993	validation_1-rmse:4.05712
[8]	validation_0-rmse:3.44842	validation_1-rmse:4.04646
[9]	validation_0-rmse:3.43595	validation_1-rmse:4.03575
[10]	validation_0-rmse:3.42405	validation_1-rmse:4.02520
[11]	validation_0-rmse:3.41343	validation_1-rmse:4.01520
[12]	validation_0-rmse:3.40218	validation_1-rmse:4.00536
[13]	validation_0-rmse:3.39166	validation_1-rmse:3.99522
[14]	validation_0-rmse:3.38103	validation_1-rmse:3.98547
[15]	validation_0-rmse:3.37075	valid

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, enable_categorical=False,
             gamma=0.6, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.01, max_delta_step=0,
             max_depth=4, min_child_weight=0, missing=nan,
             monotone_constraints='()', n_estimators=600, n_jobs=8, nthread=-1,
             num_parallel_tree=1, objective='reg:linear', predictor='auto',
             random_state=42, reg_alpha=6e-05, reg_lambda=1, scale_pos_weight=1,
             seed=27, subsample=0.7, tree_method='exact', validate_parameters=1, ...)

In [145]:
print('make predictions...')
X_train_pred_xgb      = model.predict(X_train)#, num_iteration=gbm.best_iteration_)
X_validation_pred_xgb = model.predict(X_validation)#, num_iteration=gbm.best_iteration_)
X_test_pred_xgb       = model.predict(X_test)#, num_iteration=gbm.best_iteration_)

X_submission_pred_xgb = model.predict(X_submission)#, num_iteration=gbm.best_iteration_)


X_train_pred_xgb = np.maximum(X_train_pred_xgb, 0)
X_validation_pred_xgb = np.maximum(X_validation_pred_xgb, 0)
X_test_pred_xgb = np.maximum(X_test_pred_xgb, 0)
X_submission_pred_xgb = np.maximum(X_submission_pred_xgb, 0)

df_train_rmse_xgb = mean_squared_error(y_train,X_train_pred_xgb, squared=False)
df_val_rmse_xgb   = mean_squared_error(y_validation,X_validation_pred_xgb, squared=False)
df_test_rmse_xgb  = mean_squared_error(y_test,X_test_pred_xgb, squared=False)

print('Final score mean_squared_error')
print('Score train:',df_train_rmse_xgb)
print('Score val  :',df_val_rmse_xgb)
print('Score test :',df_test_rmse_xgb)


make predictions...
Final score mean_squared_error
Score train: 2.637834447804392
Score val  : 3.279107186298613
Score test : 3.279107186298613


In [146]:
from catboost import CatBoostRegressor
from catboost import CatBoost, CatBoostClassifier, CatBoostRegressor, Pool

np.random.seed(724)

In [147]:
params = {'learning_rate': 0.3, 
          'depth': 10, 
          'l2_leaf_reg': 3, 
          #'loss_function': 'MultiRMSE', 
          #'eval_metric': 'MultiRMSE', 
          'loss_function': 'RMSE', 
          'eval_metric': 'RMSE', 
          'task_type': 'CPU', 
          'iterations': 200,
          'od_type': 'Iter', 
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bernoulli', 
          'allow_const_label': True, 
          'random_state':42
         }

In [148]:
# Label Smoothing

cat_tr = Pool(X_train, label = y_train)
cat_val = Pool(X_validation, label = y_validation)

model = CatBoostRegressor(**params)

fit_model = model.fit(cat_tr, eval_set = cat_val, early_stopping_rounds = 5, 
                      use_best_model = True, verbose = 1)

0:	learn: 2.8692318	test: 3.4731805	best: 3.4731805 (0)	total: 170ms	remaining: 33.7s
1:	learn: 2.7764978	test: 3.4045559	best: 3.4045559 (1)	total: 331ms	remaining: 32.8s
2:	learn: 2.7147348	test: 3.3437991	best: 3.3437991 (2)	total: 484ms	remaining: 31.8s
3:	learn: 2.6761923	test: 3.3251680	best: 3.3251680 (3)	total: 639ms	remaining: 31.3s
4:	learn: 2.6440147	test: 3.3143618	best: 3.3143618 (4)	total: 805ms	remaining: 31.4s
5:	learn: 2.6238934	test: 3.3045341	best: 3.3045341 (5)	total: 965ms	remaining: 31.2s
6:	learn: 2.6037749	test: 3.2992881	best: 3.2992881 (6)	total: 1.14s	remaining: 31.5s
7:	learn: 2.5864571	test: 3.2777009	best: 3.2777009 (7)	total: 1.31s	remaining: 31.5s
8:	learn: 2.5736623	test: 3.2700376	best: 3.2700376 (8)	total: 1.48s	remaining: 31.3s
9:	learn: 2.5578127	test: 3.2667838	best: 3.2667838 (9)	total: 1.63s	remaining: 30.9s
10:	learn: 2.5449194	test: 3.2616822	best: 3.2616822 (10)	total: 1.78s	remaining: 30.6s
11:	learn: 2.5313153	test: 3.2534502	best: 3.2534502

In [149]:
print('make predictions...')
X_train_pred_cat      = fit_model.predict(X_train)#, num_iteration=gbm.best_iteration_)
X_validation_pred_cat = fit_model.predict(X_validation)#, num_iteration=gbm.best_iteration_)
X_test_pred_cat       = fit_model.predict(X_test)#, num_iteration=gbm.best_iteration_)

X_submission_pred_cat = fit_model.predict(X_submission)#, num_iteration=gbm.best_iteration_)


X_train_pred_cat = np.maximum(X_train_pred_cat, 0)
X_validation_pred_cat = np.maximum(X_validation_pred_cat, 0)
X_test_pred_cat = np.maximum(X_test_pred_cat, 0)
X_submission_pred_cat = np.maximum(X_submission_pred_cat, 0)

df_train_rmse_cat = mean_squared_error(y_train,X_train_pred_cat, squared=False)
df_val_rmse_cat   = mean_squared_error(y_validation,X_validation_pred_cat, squared=False)
df_test_rmse_cat  = mean_squared_error(y_test,X_test_pred_cat, squared=False)

print('Final score mean_squared_error')
print('Score train:',df_train_rmse_cat)
print('Score val  :',df_val_rmse_cat)
print('Score test :',df_test_rmse_cat)


make predictions...
Final score mean_squared_error
Score train: 2.233688903541402
Score val  : 3.080885766937509
Score test : 3.080885766937509


In [150]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
from typing import Union
from tqdm import tqdm_notebook as tqdm
from sklearn import preprocessing
import gc
import lightgbm as lgb
from datetime import datetime, timedelta
from sklearn.model_selection import GroupKFold
from sklearn import metrics

In [151]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * 1.15)
    hess = np.where(residual < 0, 2, 2 * 1.15)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * 1.15) 
    return "custom_asymmetric_eval", np.mean(loss), False


In [152]:
X_train.head(2)

,I100,date_block_num,day_of_year,is_quarter_start,day,S103,S100,C101,is_month_end,I101,day_of_week,year,month,is_year_start,I102,is_month_start,S102,S101,I103,item_id,C100
177,0,52,7,False,7,10,0,79,False,2,6,2018,1,False,1,False,17,1,1,5,12
178,0,52,7,False,7,10,0,354,False,2,6,2018,1,False,1,False,17,1,1,10,13


In [153]:
# define random hyperparammeters
params = {
    'boosting_type': 'gbdt',
    'n_jobs': -1,
    'seed': 42,
    'learning_rate': 0.1,
    'bagging_fraction': 0.85,
    'bagging_freq': 1, 
    'colsample_bytree': 0.85,
    'colsample_bynode': 0.85,
    'min_data_per_leaf': 25,
    'metric': 'rmse',
    'num_leaves': 200,
    'lambda_l1': 0.5,
    'lambda_l2': 0.5}

params ={      
  'boosting_type': 'gbdt',
   'objective': 'tweedie',
   'tweedie_variance_power': 1.1,
    'metric': 'rmse',
    'subsample': 0.6,
    'subsample_freq': 1,
    'learning_rate': 0.02,
    'num_leaves': 512,
    'min_data_in_leaf':1024,
    'feature_fraction': 0.6,
    'max_bin': 100,
    #'n_estimators': …,   
    'boost_from_average': False,
    'verbose': -1,
    'num_threads': 12
   }    


oof = np.zeros(len(X_train))
preds = np.zeros(len(X_validation))

# GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
# kf = GroupKFold(5)
# get subgroups for each week, year pair
train_set = lgb.Dataset(X_train, y_train, 
                        categorical_feature =  ["S100","S101", "S102","S103","I100","I101","I102","I103","C100","C101"])

val_set = lgb.Dataset(X_validation,y_validation, 
                      categorical_feature =  ["S100","S101", "S102","S103","I100","I101","I102","I103","C100","C101"])


In [ ]:

# train with our custom loss function and evaluation metric
model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                  valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
                  feval = custom_asymmetric_valid)

Training until validation scores don't improve for 50 rounds
[50]	training's rmse: 2.75193	training's custom_asymmetric_eval: 8.68208	valid_1's rmse: 3.42591	valid_1's custom_asymmetric_eval: 13.4729
[100]	training's rmse: 2.48491	training's custom_asymmetric_eval: 6.99895	valid_1's rmse: 3.17369	valid_1's custom_asymmetric_eval: 11.4823
[150]	training's rmse: 2.39189	training's custom_asymmetric_eval: 6.42925	valid_1's rmse: 3.09078	valid_1's custom_asymmetric_eval: 10.8284
[200]	training's rmse: 2.34248	training's custom_asymmetric_eval: 6.13744	valid_1's rmse: 3.05726	valid_1's custom_asymmetric_eval: 10.5581
[250]	training's rmse: 2.30783	training's custom_asymmetric_eval: 5.94154	valid_1's rmse: 3.03909	valid_1's custom_asymmetric_eval: 10.4084
[300]	training's rmse: 2.27899	training's custom_asymmetric_eval: 5.7837	valid_1's rmse: 3.02362	valid_1's custom_asymmetric_eval: 10.2877
[350]	training's rmse: 2.25559	training's custom_asymmetric_eval: 5.65866	valid_1's rmse: 3.01262	val

In [ ]:
print('make predictions...')
X_train_pred      = model.predict(X_train)#, num_iteration=gbm.best_iteration_)
X_validation_pred = model.predict(X_validation)#, num_iteration=gbm.best_iteration_)
X_test_pred       = model.predict(X_test)#, num_iteration=gbm.best_iteration_)

X_submission_pred = model.predict(X_submission)#, num_iteration=gbm.best_iteration_)


X_train_pred = np.maximum(X_train_pred, 0)
X_validation_pred = np.maximum(X_validation_pred, 0)
X_test_pred = np.maximum(X_test_pred, 0)
X_submission_pred = np.maximum(X_submission_pred, 0)

df_train_rmse = mean_squared_error(y_train,X_train_pred, squared=False)
df_val_rmse   = mean_squared_error(y_validation,X_validation_pred, squared=False)
df_test_rmse  = mean_squared_error(y_test,X_test_pred, squared=False)

print('Final score mean_squared_error')
print('Score train:',df_train_rmse)
print('Score val  :',df_val_rmse)
print('Score test :',df_test_rmse)


In [ ]:

# define lgbm simple model
def run_lgb(data, features, cat_features):
    
    # reset_index
    data.reset_index(inplace = True, drop = True)
    
    # going to evaluate with the last 28 days
    x_train = data[data['date'] <= '2016-04-24']
    y_train = x_train['demand']
    test = data[data['date'] >= '2016-04-25']

    # define random hyperparammeters
    params = {
        'boosting_type': 'gbdt',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.1,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        'min_data_per_leaf': 25,
        'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5}
    
    oof = np.zeros(len(x_train))
    preds = np.zeros(len(test))
    
    # GroupKFold by week, month to avoid leakage and overfitting (not entirely sure xD)
    kf = GroupKFold(5)
    # get subgroups for each week, year pair
    group = x_train['week'].astype(str) + '_' + x_train['year'].astype(str)
    for fold, (trn_idx, val_idx) in enumerate(kf.split(x_train, y_train, group)):
        print(f'Training fold {fold + 1}')
        train_set = lgb.Dataset(x_train.iloc[trn_idx][features], y_train.iloc[trn_idx], 
                                categorical_feature = cat_features)
        val_set = lgb.Dataset(x_train.iloc[val_idx][features], y_train.iloc[val_idx], 
                              categorical_feature = cat_features)
        
        # train with our custom loss function and evaluation metric
        model = lgb.train(params, train_set, num_boost_round = 10000, early_stopping_rounds = 50, 
                          valid_sets = [train_set, val_set], verbose_eval = 50, fobj = custom_asymmetric_train, 
                          feval = custom_asymmetric_valid)
    
        # predict oof
        oof[val_idx] = model.predict(x_train.iloc[val_idx][features])

        # predict test
        preds += model.predict(test[features]) / 5
        
        print('-'*50)
        print('\n')
        
    oof_rmse = np.sqrt(metrics.mean_squared_error(y_train, oof))
    print(f'Our out of folds rmse is {oof_rmse}')
        
    test = test[['id', 'date', 'demand']]
    test['demand'] = preds
    return test


# this is the main function that will run our entire program
def train_and_evaluate():
    
    # read data
    print('Reading our data...')
    data, submission = read_data()
    
    data['date'] = pd.to_datetime(data['date'])
    # get amount of unique days in our data
    days = abs((data['date'].min() - data['date'].max()).days)
    # how many training data do we need to train with at least 2 years and consider lags
    need = 365 + 365 + 90 + 28
    print(f'We have {(days - 28)} days of training history')
    print(f'we have {(days - 28 - need)} days left')
    if (days - 28 - need) > 0:
        print('We have enought training data, lets continue')
    else:
        print('Get more training data, training can fail')
    
    # simple feature engineer
    print('Running simple feature engineering...')
    data = simple_fe(data)
    print('Removing first 118 days')
    # eliminate the first 118 days of our train data because of lags
    min_date = data['date'].min() + timedelta(days = 118)
    data = data[data['date'] > min_date]
    
    # define our numeric features and categorical features
    features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'sell_price', 'year', 
                'month', 'quarter', 'week', 'day', 'dayofweek', 'dayofyear', 'demand_rolling_mean_t7', 'demand_rolling_mean_t15', 'demand_rolling_mean_t30', 'demand_rolling_mean_t90',
                'demand_rolling_std_t7', 'demand_rolling_std_t15', 'demand_rolling_std_t30', 'demand_rolling_std_t90']
    
    cat_features = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 
                    'event_name_2', 'event_type_2']
    
    print('-'*50)
    print('\n')
    print(f'Training model with {len(features)} features...')
    # run lgbm model with 5 GroupKFold (subgroups by year, month)
    test = run_lgb(data, features, cat_features)
    print('Save predictions...')
    # predict
    predict(test, submission)
        
# run our program
train_and_evaluate()

In [23]:
df_sales2 = df_sales
X_train      = df_sales2[(df_sales2['DATE'] >= '2017-01-01') & (df_sales2['DATE'] < '2021-05-01')].copy()
X_validation = df_sales2[(df_sales2['DATE'] >= '2021-05-01') & (df_sales2['DATE'] < '2021-08-01')].copy()
X_test       = df_sales2[(df_sales2['DATE'] >= '2021-08-01') & (df_sales2['DATE'] < '2021-10-03')].copy() 


In [27]:
#exogenous_features = ['is_quarter_start', 'day_of_year', 'day_of_week', 'month', 'is_year_start', 'day', 'year', 'is_month_end', 'is_month_start','date_block_num']

exogenous_features = ['is_quarter_start', 'day_of_year', 'day_of_week', 'month', 'is_year_start', 'day', 'year', 'is_month_end', 'is_month_start','date_block_num']

In [30]:
from fbprophet import Prophet

model_fbp = Prophet()
for feature in exogenous_features:
    model_fbp.add_regressor(feature)

In [31]:
model_fbp.fit(X_train.rename(columns={"DATE": "ds", "QTT": "y"}))


INFO:fbprophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -30.8797
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Exception: normal_lpdf: Scale parameter is 0, but must be > 0!  (in 'unknown file name' at line 136)

Error evaluating model log probability: Non-finite gradient.
Error evaluating model log probability: Non-finite gradient.

      99   1.21782e+07   0.000114789        698776      0.3823           1      130   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199   1.21783e+07   1.59786e-05        764715           1           1      252   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299   1.21786e+07   7.25083e-05   1.71256e+06           1           1      371   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
   

In [1]:
print('make predictions...')
#X_train_pred_cat      = model_fbp.predict(X_train.rename(columns={"DATE": "ds"})).yhat.values
X_validation_pred_cat = model_fbp.predict(X_validation.rename(columns={"DATE": "ds"})).yhat.values
X_test_pred_cat       = model_fbp.predict(X_test.rename(columns={"DATE": "ds"})).yhat.values

X_submission_pred_cat = model_fbp.predict(df_submission_sample.rename(columns={"DATE": "ds"})).yhat.values

#y_train = X_train['QTT']
y_validation = X_validation['QTT']
y_test = X_test['QTT']


#X_train_pred_cat = np.maximum(X_train_pred_cat, 0)
X_validation_pred_cat = np.maximum(X_validation_pred_cat, 0)
X_test_pred_cat = np.maximum(X_test_pred_cat, 0)
X_submission_pred_cat = np.maximum(X_submission_pred_cat, 0)

#df_train_rmse_cat = mean_squared_error(y_train,X_train_pred_cat, squared=False)
df_val_rmse_cat   = mean_squared_error(y_validation,X_validation_pred_cat, squared=False)
df_test_rmse_cat  = mean_squared_error(y_test,X_test_pred_cat, squared=False)

print('Final score mean_squared_error')
#print('Score train:',df_train_rmse_cat)
print('Score val  :',df_val_rmse_cat)
print('Score test :',df_test_rmse_cat)

make predictions...


NameError: name 'model_fbp' is not defined

In [24]:
from pmdarima import auto_arima

In [37]:
df_sales2 = df_sales
X_train      = df_sales2[(df_sales2['DATE'] >= '2017-01-01') & (df_sales2['DATE'] < '2021-05-01')].copy()
X_validation = df_sales2[(df_sales2['DATE'] >= '2021-05-01') & (df_sales2['DATE'] < '2021-08-01')].copy()
X_test       = df_sales2[(df_sales2['DATE'] >= '2021-08-01') & (df_sales2['DATE'] < '2021-10-03')].copy() 

y_train = X_train['QTT']
X_train = X_train[features_names]

y_validation = X_validation['QTT']
X_validation = X_validation[features_names]

y_test = X_test['QTT']
X_test = X_test[features_names]

y_submission = df_submission_sample['QTT']
X_submission = df_submission_sample[features_names]


print('X_train      ',X_train.shape)
print('X_validation ',X_validation.shape)
print('X_test       ',X_test.shape)
print('X_submission ',X_submission.shape)

X_train       (2511660, 21)
X_validation  (142194, 21)
X_test        (95515, 21)
X_submission  (69121, 21)


In [38]:
X_train.head()

,I100,date_block_num,day_of_year,is_quarter_start,day,S103,S100,C101,is_month_end,I101,...,year,month,is_year_start,I102,is_month_start,S102,S101,I103,item_id,C100
0,0,0,8,False,8,10,0,76,False,2,...,2017,1,False,1,False,17,1,1,4,12
1,0,0,8,False,8,10,0,149,False,2,...,2017,1,False,1,False,17,1,1,0,12
2,0,0,8,False,8,10,0,256,False,2,...,2017,1,False,1,False,17,1,1,2,12
3,0,0,8,False,8,10,0,354,False,2,...,2017,1,False,1,False,17,1,1,3,12
4,0,0,8,False,8,10,0,149,False,2,...,2017,1,False,1,False,17,1,1,6,13


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,8))
plt.plot(X_test_pred,'b', alpha=0.7)
plt.plot(y_test.values,'r', alpha=0.4)
plt.legend(["prediction", "real"], loc ="upper right")

plt.show()

In [ ]:
df_submission_sample.reset_index(drop=True,inplace=True)
df_submission_sample['QTT'] = X_submission_pred
df_submission_sample.head()

In [ ]:
filename_submission= 'submission_NOFILL_good_'+str(np.round(df_test_rmse,4))+'_local.csv'
save = False
if save:
    print('saving..')
    print(filename_submission)
    df_submission_sample[['ID','QTT']].to_csv(os.path.join(PATH_RESULTS,'submissions','no_fill',filename_submission),index=None)

In [ ]:
result = pd.read_csv(os.path.join(PATH_RESULTS,'submissions','submission_2.1046_local.csv'))

In [ ]:
df_test['QTT_reference']= result['QTT'].copy()
df_test[['QTT_reference','QTT']][-20:]

In [ ]:
fig = plt.figure(figsize=(20,8))
plt.plot(df_submission_sample['QTT'],'b', alpha=0.7)
plt.plot(result['QTT'],'r', alpha=0.4)
plt.legend(["prediction", "real"], loc ="upper right")
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

difference = mean_squared_error(df_test['QTT'],result['QTT'], squared=False)

print('Final score mean_squared_error')
print('Score train:',difference)